In [ ]:
from bs4 import BeautifulSoup
import csv
import pandas as pd
import requests
from datetime import datetime, timedelta, date

## This is the basic for getting the HTML and parsing it

In [ ]:
## Here we select the link for the jobs page for NCCI
link = "https://ncciraqjobs.com/index.php/jobseeker-control-panel"

In [ ]:
def get_soup (url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text,"html.parser")
    return soup

In [ ]:
records = []
def get_record(card):
    job_title = card.find("a",{"class":"js_job_title"}).text.strip()
    ngo = card.find("a",{"class":"js_job_data_2_company_link"}).text.strip()
    job_location = card.find("span",{"class":"js_job_data_location_value"}).text.strip()
    export_date = date.today()
    post_date = card.find("span",{"class":"js_job_posted"}).text.strip()[8:]

    if post_date == "Today":
        post_date = date.today()
    else:
        post_date = export_date - timedelta(int(post_date.split()[0]))

    job_url = "https://ncciraqjobs.com" + card.find("a",{"class":"js_job_data_2_company_link"}).get("href").strip()
    
    record = [job_title, ngo, job_location, export_date, post_date, job_url]
    return record

In [ ]:
def get_cards():
    cards = get_soup(link).find_all("div",{"class":"js_job_main_wrapper"})
    return cards

In [ ]:
len(records)

In [ ]:
while True:
    try:
        soup = get_soup(link)
        cards = get_cards()
        for card in cards: 
            record = get_record(card)
            records.append(record)
        link = "https://ncciraqjobs.com" + soup.find("a",{"aria-label":"Go to next page"}).get("href")
        print(link)
    except:
        break

In [ ]:
data = pd.DataFrame(records, index=None, columns=["Job Title", "Organization", "Job Location", "Export Date", "Post Date",
                                                  "Job URL"])

In [ ]:
data.to_csv(r"Enter designated path here or just leave it default as data.csv", index=False, encoding="utf-8-sig")